In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn


from training.model_training import NN, get_loaders, get_accuracy
from training.cnn import CNN, get_loaders_cnn

from interface.interfacer import PseudoHardwareNN, PseudoHardwareCNN
from interface.IOTransfer import quantize

from hardware.crossbars import SimpleCrossbar, SimpleCrossbar2, DifferentialCrossbar
from evaluator import compare_models

### NN Evaluation

In [ ]:
model_path = "training/model.pth"
l1_model_path = "training/model_l1reg.pth"

device = "cpu" # TODO need to fix for cuda and mps (atleast the software model)

ann = NN()                                        
# ann_reg = NN()
# hwnn = PseudoHardwareNN(hardware_multiplier=SimpleCrossbar2, 
#                                         input_quantization=(True, 0.01),
#                                         weight_quantization=(True, 0.05, [-1,1]),
#                                         output_quantization=(True, 0.01),
#                                         weight_variance=(True, 0.05),
#                                         inline_resistances=(True, (1e-4, 1e-4)),
#                                         verbose=False)

# hwnn_l1reg = PseudoHardwareNN(hardware_multiplier=SimpleCrossbar2, 
#                                         input_quantization=(True, 0.01),
#                                         weight_quantization=(True, 0.05, [-1,1]),
#                                         output_quantization=(True, 0.01),
#                                         weight_variance=(True, 0.05),
#                                         inline_resistances=(True, (1e-4, 1e-4)),
#                                         verbose=False)


# hwnn2 = PseudoHardwareNN(hardware_multiplier=SimpleCrossbar2, 
#                                         input_quantization=(True, 0.01),
#                                         weight_quantization=(True, 0.05, [-1,1]),
#                                         output_quantization=(True, 0.01),
#                                         inline_resistances=(True, (1e-4, 1e-4)))
# hwnn3 = PseudoHardwareNN(hardware_multiplier=DifferentialCrossbar,
#                                         input_quantization=(True, 0.01),
#                                         weight_quantization=(True, 0.05, [-1,1]),
#                                         output_quantization=(True, 0.01),
#                                         inline_resistances=(True, (1e-4, 1e-4)))


models = [ann] #, hwnn3]

for model in models:
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

# ann_reg.load_state_dict(torch.load(l1_model_path))
# ann_reg.to(device)
# ann_reg.eval()

# models.append(ann_reg)

# # hwnn_l1reg.load_state_dict(torch.load(l1_model_path))
# # hwnn_l1reg.to(device)
# # hwnn_l1reg.eval()

# # models.append(hwnn_l1reg)

# # dataset
# test_data = get_loaders(batch_size=1, train=False, dataset_path="datasets/")
max_samples = None

compare_models(models, names=['ann', 'hwnn', 'ann_reg'],max_samples=max_samples)


In [ ]:
# plt.bar(["Software", "Ideal Hardware", "QSI"], [software_accuracy,hardware_accuracy, hardware_quantized_accuracy])
# plt.hlines(software_accuracy, -0.5, 2.5, colors="r", linestyles="dashed", label="Software Baseline")
# plt.ylabel("Accuracy")
# plt.title("Model Accuracy")
# plt.legend()
# plt.show()

### Weight Viz

In [ ]:
# plot weight distributions

def plot_weights(model, layer_idx=None):

    if layer_idx is not None:
        for i, layer in enumerate(model.children()):
            if i == layer_idx:
                mylayer = layer
                break

        weights = mylayer.weight.detach().numpy().flatten()
        plt.hist(weights, bins=100)
        plt.title(f"Layer {layer_idx} weights distribution")
        plt.show()

    # side by side comparison
    else:
        total_layers = len(list(model.children()))
        fig, axs = plt.subplots(1, total_layers, figsize=(20, 5))
        

        for i, layer in enumerate(model.children()):

            weights = layer.weight.detach().numpy().flatten()
            total_weights = len(weights)


            color_list = ["blue", "red", "green", "orange", "purple", "brown", "pink", "gray", "cyan", "magenta"]
            axs[i].hist(weights, bins=100, color=color_list[i])
            # axs[i].grid()

            axs[i].set_xlabel("Weight Value", fontsize=15)
            axs[i].set_ylabel("Frequency", fontsize=15)
            axs[i].set_title(f"Layer {i}; {total_weights} weights", fontsize=15)

            # tick size
            axs[i].tick_params(axis='both', which='major', labelsize=12)
            axs[i].tick_params(axis='both', which='minor', labelsize=10)



        


        # plt.savefig("weight_distributions.eps", dpi = 600, bbox_inches='tight')
        plt.show()

plot_weights(ann)

In [ ]:
def plot_weights(model, layer_idx=None):

    if layer_idx is not None:
        for i, layer in enumerate(model.children()):
            if i == layer_idx:
                mylayer = layer
                break

        weights = quantize(mylayer.weight.detach().numpy().flatten(), 0.05, [-1,1])

        
        plt.hist(weights, bins=100)
        plt.title(f"Layer {layer_idx} weights distribution")
        plt.show()

    # side by side comparison
    else:
        total_layers = len(list(model.children()))
        fig, axs = plt.subplots(1, total_layers, figsize=(20, 5))

        for i, layer in enumerate(model.children()):
            weights = quantize(layer.weight.detach().numpy().flatten(), 0.05, [-1,1]) + np.random.normal(0, 0.01, len(layer.weight.detach().numpy().flatten()))
            total_weights = len(weights)

            color_list = ["blue", "red", "green", "orange", "purple", "brown", "pink", "gray", "cyan", "magenta"]
            axs[i].hist(weights, bins=100, color=color_list[i])
            # axs[i].grid()

            axs[i].set_xlabel("Weight Value", fontsize=15)
            axs[i].set_ylabel("Frequency", fontsize=15)
            axs[i].set_title(f"Layer {i}; {total_weights} weights", fontsize=15)

            # tick size
            axs[i].tick_params(axis='both', which='major', labelsize=12)
            axs[i].tick_params(axis='both', which='minor', labelsize=10)


        # plt.savefig("weight_distributions_quantized_variance.eps", dpi = 600, bbox_inches='tight')        
        plt.show()

plot_weights(ann)

In [ ]:
import torch
import numpy as np

def conv2mm(kernel, input_tensor):
    """
    Returns the flattened kernel and corresponding 2D matrix of the input tensor
    such that the convolution operation can be represented as a matrix multiplication
    """

    input_tensor = input_tensor.flatten()

    # toeplitz matrix
    kernel = kernel.flatten()
    kernel_size = len(kernel)
    input_size = len(input_tensor)
    output_size = input_size - kernel_size + 1

    toeplitz_matrix = np.zeros((output_size, input_size))

    for i in range(output_size):
        toeplitz_matrix[i, i:i+kernel_size] = kernel

    return torch.tensor(toeplitz_matrix), input_tensor

    

## check correctness of conv2mm

my_kernel = torch.tensor([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
inputImage = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]])

toeplitz_matrix, input_tensor = conv2mm(my_kernel, inputImage)

print(np.dot(toeplitz_matrix, input_tensor))
# print(np.matmul(toeplitz_matrix, input_tensor))
print(torch.nn.functional.conv2d(inputImage.unsqueeze(0).unsqueeze(0), my_kernel.unsqueeze(0).unsqueeze(0)))


### FFNN Evaluation


In [ ]:
model_path = "training/model.pth"


io_resolution = np.array([0.1]*10)
weight_quantisation_list = 2**np.array([8])
wtqt_list = 1/weight_quantisation_list
weight_variability_list = np.array([1e-1, 1e-2, 1e-3, 1e-4])


print("io,wtqt,wtvt,acc")
for ioqt in io_resolution:
    for wtqt in wtqt_list:
        models = []
        for wtvt in weight_variability_list:
            hwnn = PseudoHardwareNN(input_quantization=(True, ioqt),
                                    weight_quantization=(True, wtqt, [-1,1]),
                                    output_quantization=(True, ioqt),
                                    weight_variance=(True, wtvt),
                                    verbose=False)
                                    
            hwnn.load_state_dict(torch.load(model_path))
            hwnn.eval()
            models.append(hwnn)
        
        results = compare_models(models, np.arange(len(weight_variability_list)), max_samples=2000, get_accuracy=True, verbose=False)
        for i, result in enumerate(results):
            print(f"{ioqt}, {np.log2(1/wtqt)}, {weight_variability_list[i]}, {result}")



In [ ]:
model_path = "training/model.pth"


io_resolution = np.array([0.1]*5)
weight_quantisation_list = 2**np.array([2,4,6,8,10])
wtqt_list = 1/weight_quantisation_list
weight_variability_list = np.array([1e-1, 1e-2, 1e-3, 1e-4])


print("io,wtqt,wtvt,acc")
for ioqt in io_resolution:
    for wtqt in wtqt_list:
        models = []
        for wtvt in weight_variability_list:
            hwnn = PseudoHardwareNN(input_quantization=(True, ioqt),
                                    weight_quantization=(True, wtqt, [-1,1]),
                                    output_quantization=(True, ioqt),
                                    weight_variance=(True, wtvt),
                                    verbose=False)
                                    
            hwnn.load_state_dict(torch.load(model_path))
            hwnn.eval()
            models.append(hwnn)
        
        results = compare_models(models, np.arange(len(weight_variability_list)), max_samples=2000, get_accuracy=True, verbose=False)
        for i, result in enumerate(results):
            print(f"{ioqt}, {np.log2(1/wtqt)}, {weight_variability_list[i]}, {result}")



In [ ]:
model_path = "training/model.pth"


io_resolution = np.array([0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1])
weight_quantisation_list = 2**np.array([6,6,6,6,6])
wtqt_list = 1/weight_quantisation_list
weight_variability_list = np.array([1e-1, 1e-2, 1e-3, 1e-4])


print("io, wtqt, wtvt, acc")
for ioqt in io_resolution:
    for wtqt in wtqt_list:
        models = []
        for wtvt in weight_variability_list:
            hwnn = PseudoHardwareNN(input_quantization=(True, ioqt),
                                    weight_quantization=(True, wtqt, [-1,1]),
                                    output_quantization=(True, ioqt),
                                    weight_variance=(True, wtvt),
                                    verbose=False)
                                    
            hwnn.load_state_dict(torch.load(model_path))
            hwnn.eval()
            models.append(hwnn)
        
        results = compare_models(models, np.arange(len(weight_variability_list)), max_samples=2000, get_accuracy=True, verbose=False)
        for i, result in enumerate(results):
            print(f"{ioqt}, {np.log2(1/wtqt)}, {weight_variability_list[i]}, {result}")



In [ ]:
model_path = "training/model.pth"
ann = NN()

ann.load_state_dict(torch.load(model_path))
ann.eval()

models = [ann, ]
names = ["Software FFNN", ]

test_data = get_loaders(batch_size=1, train=False, dataset_path="datasets/")
max_samples = None


compare_models(models, names, max_samples=max_samples, dataset=test_data)

### CNN Evaluation

In [ ]:
from interface.interfacer import PseudoHardwareNN, PseudoHardwareCNN
cnn_path = "training/cnn_weights.pth"
# cnn = CNN()
ioqt = 1e-5
hwcnn = PseudoHardwareCNN(input_quantization=(True, ioqt),
                        weight_quantization=(False, 1, [-1,1]),
                        output_quantization=(True, ioqt),
                        weight_variance=(True, 0.001))

# cnn.load_state_dict(torch.load(cnn_path))
hwcnn.load_state_dict(torch.load(cnn_path))
# cnn.eval()
hwcnn.eval()

models = [hwcnn]
names = ["CNN"]

test_data = get_loaders_cnn(batch_size=1, train=False)
max_samples = None


compare_models(models, names, max_samples=max_samples, dataset=test_data, verbose=True)

In [9]:
cnn_path = "training/cnn_weights.pth"
from interface.interfacer import PseudoHardwareCNN

io_resolution = np.array([0.1]*10)
weight_quantisation_list = 2**np.array([10])
wtqt_list = 1/weight_quantisation_list
weight_variability_list = np.array([1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125])

test_data = get_loaders_cnn(batch_size=1, train=False)
max_samples = None


print("io,wtqt,wtvt,acc")
for ioqt in io_resolution:
    for wtqt in wtqt_list:
        models = []
        for wtvt in weight_variability_list:
            hwcnn = PseudoHardwareCNN(input_quantization=(False, ioqt),
                                    weight_quantization=(False, wtqt, [-1,1]),
                                    output_quantization=(False, ioqt),
                                    weight_variance=(True, wtvt),)
                                    
            hwcnn.load_state_dict(torch.load(cnn_path))
            hwcnn.eval()
            models.append(hwcnn)
        
        # results = compare_models(models, np.arange(len(weight_variability_list)), max_samples=2000, get_accuracy=True, verbose=False)
        results = compare_models(models, np.arange(len(weight_variability_list)), max_samples=max_samples, dataset=test_data, get_accuracy=True, verbose=False)
        for i, result in enumerate(results):
            print(f"{ioqt}, {np.log2(1/wtqt)}, {weight_variability_list[i]}, {result}")

Files already downloaded and verified
io,wtqt,wtvt,acc
0.1, 10.0, 1.0, 0.1107
0.1, 10.0, 0.5, 0.1049
0.1, 10.0, 0.25, 0.1121
0.1, 10.0, 0.125, 0.2063
0.1, 10.0, 0.0625, 0.4444
0.1, 10.0, 0.03125, 0.5641
0.1, 10.0, 0.015625, 0.5802
0.1, 10.0, 0.0078125, 0.5891
0.1, 10.0, 1.0, 0.0936
0.1, 10.0, 0.5, 0.0899
0.1, 10.0, 0.25, 0.117
0.1, 10.0, 0.125, 0.2313
0.1, 10.0, 0.0625, 0.4437
0.1, 10.0, 0.03125, 0.5682
0.1, 10.0, 0.015625, 0.5849
0.1, 10.0, 0.0078125, 0.5881
0.1, 10.0, 1.0, 0.1017
0.1, 10.0, 0.5, 0.0981
0.1, 10.0, 0.25, 0.1146
0.1, 10.0, 0.125, 0.2235
0.1, 10.0, 0.0625, 0.4545
0.1, 10.0, 0.03125, 0.5483
0.1, 10.0, 0.015625, 0.5748
0.1, 10.0, 0.0078125, 0.5846
0.1, 10.0, 1.0, 0.0886
0.1, 10.0, 0.5, 0.0866
0.1, 10.0, 0.25, 0.1093
0.1, 10.0, 0.125, 0.2129
0.1, 10.0, 0.0625, 0.4224
0.1, 10.0, 0.03125, 0.5476
0.1, 10.0, 0.015625, 0.5858
0.1, 10.0, 0.0078125, 0.5874
0.1, 10.0, 1.0, 0.0919
0.1, 10.0, 0.5, 0.0998
0.1, 10.0, 0.25, 0.0986
0.1, 10.0, 0.125, 0.1568
0.1, 10.0, 0.0625, 0.4076
0.1, 